# Assignment 4a - Variational Auto-Encoders
## Deep Learning Course - Vrije Universiteit Amsterdam, 2022

#### Instructions on how to use this notebook:

This notebook is hosted on Google Colab. To be able to work on it, you have to create your own copy. Go to *File* and select *Save a copy in Drive*.

You can also avoid using Colab entirely, and download the notebook to run it on your own machine. If you choose this, go to *File* and select *Download .ipynb*.

The advantage of using Colab is that you can use a GPU. You can complete this assignment with a CPU, but it will take a bit longer. Furthermore, we encourage you to train using the GPU not only for faster training, but also to get experience with this setting. This includes moving models and tensors to the GPU and back. This experience is very valuable because for various models and large datasets (like large CNNs for ImageNet, or Transformer models trained on Wikipedia), training on GPU is the only feasible way.

The default Colab runtime does not have a GPU. To change this, go to *Runtime - Change runtime type*, and select *GPU* as the hardware accelerator. The GPU that you get changes according to what resources are available at the time, and its memory can go from a 5GB, to around 18GB if you are lucky. If you are curious, you can run the following in a code cell to check:

```sh
!nvidia-smi
```

Note that despite the name, Google Colab does  not support collaborative work without issues. When two or more people edit the notebook concurrently, only one version will be saved. You can choose to do group programming with one person sharing the screen with the others, or make multiple copies of the notebook to work concurrently.

**Submission:** Upload your notebook in .ipynb format to Canvas. The code and answers to the questions in the notebook are sufficient, no separate report is expected. 

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## Introduction

In this assignment, we are going to implement a Variational Auto-Encoder (VAE). A VAE is a likelihood-based deep generative model that consists of a stochastic encoder (a variational posterior over latent variables), a stochastic decoder, and a marginal distribution over latent variables (a.k.a. a prior). The model was originally proposed in two concurrent papers:
- [Kingma, D. P., & Welling, M. (2013). Auto-encoding variational bayes. arXiv preprint arXiv:1312.6114.](https://arxiv.org/abs/1312.6114)
- [Rezende, Danilo Jimenez, Shakir Mohamed, and Daan Wierstra. "Stochastic backpropagation and approximate inference in deep generative models." International conference on machine learning. PMLR, 2014.](https://proceedings.mlr.press/v32/rezende14.html)

You can read more about VAEs in Chapter 4 of the following book:
- [Tomczak, J.M., "Deep Generative Modeling", Springer, 2022](https://link.springer.com/book/10.1007/978-3-030-93158-2)

In particular, the goals of this assignment are the following:

- Understand how VAEs are formulated
- Implement components of VAEs using PyTorch
- Train and evaluate a model for image data

### Theory behind VAEs

VAEs are latent variable models trained with variational inference. In general, the latent variable models define the following generative process:
\begin{align}
1.\ & \mathbf{z} \sim p_{\lambda}(\mathbf{z}) \\
2.\ & \mathbf{x} \sim p_{\theta}(\mathbf{x}|\mathbf{z})
\end{align}

In plain words, we assume that for observable data $\mathbf{x}$, there are some latent (hidden) factors $\mathbf{z}$. Then, the training objective is log-likelihood function of the following form:
$$
\log p_{\vartheta}(\mathbf{x})=\log \int p_\theta(\mathbf{x} \mid \mathbf{z}) p_\lambda(\mathbf{z}) \mathrm{d} \mathbf{z} .
$$

The problem here is the intractability of the integral if the dependencies between random variables $\mathbf{x}$ and $\mathbf{z}$ are non-linear and/or the distributions are non-Gaussian.

By introducing variational posteriors $q_{\phi}(\mathbf{z}|\mathbf{x})$, we get the following lower bound (the Evidence Lower Bound, ELBO):
$$
\log p_{\vartheta}(\mathbf{x}) \geq \mathbb{E}_{\mathbf{z} \sim q_\phi(\mathbf{z} \mid \mathbf{x})}\left[\log p_\theta(\mathbf{x} \mid \mathbf{z})\right]-\mathrm{KL}\left(q_\phi(\mathbf{z} \mid \mathbf{x}) \| p_\lambda(\mathbf{z})\right) .
$$

Note that we want to *maximize* this objective, therefore, in the code you are going to have to implement NELBO (negative ELBO) as a loss function (i.e., a minimization task). 

## IMPORTS

In [2]:
# DO NOT REMOVE!
import os

import numpy as np
import matplotlib.pyplot as plt

import torch

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision.datasets import MNIST

In [3]:
# Check if GPU is available and determine the device
if torch.backends.mps.is_available():
  device = 'mps'  # For Apple Silicon (M1/M2/M3)
elif torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

print(f'The available device is {device}')
# mount drive: WE NEED IT FOR SAVING IMAGES!
# Note: Not needed for local,ininppeassssssass

The available device is mps


In [4]:
# PLEASE CHANGE IT TO YOUR OWN GOOGLE DRIVE!
images_dir = './results/'
os.makedirs(images_dir,exist_ok=True)

## Auxiliary functions

Let us define some useful log-distributions:

In [5]:
# DO NOT REMOVE
PI = torch.from_numpy(np.asarray(np.pi))
EPS = 1.e-5


def log_categorical(x, p, num_classes=256, reduction=None, dim=None):
    x_one_hot = F.one_hot(x.long(), num_classes=num_classes)
    log_p = x_one_hot * torch.log(torch.clamp(p, EPS, 1. - EPS))
    if reduction == 'avg':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_bernoulli(x, p, reduction=None, dim=None):
    pp = torch.clamp(p, EPS, 1. - EPS)
    log_p = x * torch.log(pp) + (1. - x) * torch.log(1. - pp)
    if reduction == 'avg':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_normal_diag(x, mu, log_var, reduction=None, dim=None):
    D = x.shape[1]
    log_p = -0.5 * D * torch.log(2. * PI) - 0.5 * log_var - 0.5 * torch.exp(-log_var) * (x - mu)**2.
    if reduction == 'avg':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_standard_normal(x, reduction=None, dim=None):
    D = x.shape[1]
    log_p = -0.5 * D * torch.log(2. * PI) - 0.5 * x**2.
    if reduction == 'avg':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p

## Implementing VAEs

The goal of this assignment is to implement four classes:
- `Encoder`: this class implements the encoder (variational posterior), $q_{\phi}(\mathbf{z}|\mathbf{x})$.
- `Decoder`: this class implements the decoded (the conditional likelihood), $p_{\theta}(\mathbf{x}|\mathbf{z})$.
- `Prior`: this class implements the marginal over latents (the prior), $p_{\lambda}(\mathbf{z})$.
- `VAE`: this class combines all components.

#### Question 0: (3 pt) 
**Fully-connected Neural Networks (MLPs) or Convolutional Neural Networks**

This is not a real question but rather a comment. You are asked to implement your VAE using fully connected neural networks (MLPs) or convolutional neural networks (ConvNets). 

There is a difference in grading of this assignment based on your decision:
- **If you decide to implement your VAE with MLPs and the model works properly, you get 1 pt.**
- **If you decide to implement your VAE with ConvNets and the model works properly, you get 3 pts.**

### Encoder
We start with `Encoder`. Please remember that we assume the Gaussian variational posterior with a diagonal covariance matrix.

Feel free to add other methods to the class as well as arguments to the class initialization.

In [6]:

# YOUR CODE GOES HERE
# NOTE: The class must containt the following functions: 
# (i) reparameterization
# (ii) sample
# Moreover, forward must return the log-probability of variational posterior for given x, i.e., log q(z|x)

class Encoder(nn.Module):
    def __init__(self,encoder_net:torch.nn.Module):
        super(Encoder, self).__init__()

        #Will let it like this for now . Probably changing to Sequential later.
        #By convention powers of 2 (2**n) are mostly used for defining a layer's size.
        # self.k_dim = 512 
        # self.g_dim = 256

        # self.fc1 = nn.Linear(input_dim,self.k_dim)
        # self.fc2 = nn.Linear(self.k_dim,self.g_dim)
        # self.fc_mu = nn.Linear(self.g_dim,bottleneck_dim)
        # self.fc_log_var = nn.Linear(self.g_dim,bottleneck_dim)
        self.encoder = encoder_net # The Encoder Network . Will be passed as an argument .

    @staticmethod
    def reparameterization(mu, log_var):

        sigma =  torch.exp(0.5 * log_var) # takes the log_var and transforms it to standard deviation .
        # e = torch.normal(mean=0.0, std=1.0, size=sigma.shape)
        e = torch.randn_like(sigma) # error sampled from a standard Gaussian distr. as the suggested in the book.

        z =mu + sigma * e 
        return z
    
    
    def encode(self, x):
        
        
        h_e = self.encoder(x) # pass through the network
        mu_e ,log_var_e = torch.chunk(h_e,2,dim=1) #take the output and split it to mu and log_var .

        return mu_e,log_var_e


    def sample(self, x=None, mu_e=None, log_var_e=None):
        """
        Sample from the encoder. 
        If x is given (not equal to None), then copmute variational posterior distribution q(z|x) and sample from it.
        Otherwise, use `mu_e` and `log_var_e` as parameter of the variational posterior and sample from it.

        x: torch.tensor, with dimensionality (mini-batch, x_dim)
             a mini-batch of data points
        mu_e: torch.tensor, with dimensionality (mini-batch, x_dim)
             mean vector of the variational posterior
        log_var_e: torch.tensor, with dimensionality (mini-batch, x_dim)
             log variance of the variational posterior
        return: z: torch.tensor, with dimensionality (mini-batch, z_dim)
        """
        if x is not None:
            mu,log_var = self.encode(x)
            return self.reparameterization(mu=mu,log_var=log_var)

        else:
        
            return self.reparameterization(mu = mu_e,log_var=log_var_e)
        


    def forward(self, x,type = 'log_prob'):
        """
        Compute log-probability of variational posterior for given x, i.e., log q(z|x)
        x: torch.tensor, with dimensionality (mini-batch, x_dim)
             a mini-batch of data points
        """
        if type == 'log_prob':
            mu,log_var = self.encode(x)
            z = self.sample(mu_e=mu,log_var_e=log_var)

            return log_normal_diag(z,mu,log_var)
        else :
            return self.sample(x)

Please answer the following questions:


#### Question 1 (0.5 pt)

Please explain the reparameterization trick and provide a mathematical formula.

ANSWER :

$$
\mathbf{z} = \mathbf{\mu} + \mathbf{\sigma} \odot \mathbf{\epsilon}
$$


#### Question 2 (0.25 pt)

Please write down mathematically the log-probability of the encoder (variational posterior).

ANSWER: [Please fill in]

### Decoder

The decoder is the conditional likelihood, i.e., $p(\mathbf{x}|\mathbf{z})$. Please remember that we must decide on the form of the distribution (e.g., Bernoulli, Gaussian, Categorical). Please discuss it with a TA or a lecturer if you are in doubt.

In [7]:
# YOUR CODE GOES HERE
# NOTE: The class must containt the following function: 
# (i) sample
# Moreover, forward must return the log-probability of the conditional likelihood function for given z, i.e., log p(x|z)

class Decoder(nn.Module):
    def __init__(self,decoder_net : torch.nn.Module ):
        super(Decoder, self).__init__()

        # self.k_dim = 512
        # self.g_dim = 256
        self.decoder = decoder_net # The decoder netwrok . Will be passed as an argument.
        # self.fc1 = nn.Linear(bottleneck_dim, self.g_dim)
        # self.fc2 = nn.Linear(self.g_dim, self.k_dim)
        # self.fc3 = nn.Linear(self.k_dim, input_dim)

    def decode(self,z):
        
        h_d = self.decoder(z) # pass through the network layers.
        d_prob = torch.sigmoid(h_d) # sigmoid is used to output a probability in (0,1)
        return d_prob

    def sample(self, z):
        """
        For a given latent code compute parameters of the conditional likelihood 
        and sample x ~ p(x|z)

        z: torch.tensor, with dimensionality (mini-batch, z_dim)

        return:
        x: torch.tensor, with dimensionality (mini-batch, x_dim)
        """
        x = self.decode(z) 
        
        x_coin_flip = torch.bernoulli(x) #biased coin flip 
        return x_coin_flip

    def forward(self, z, x=None,type = 'log_prob'):
        """
        Compute the log probability: log p(x|z). 
        z: torch.tensor, with dimensionality (mini-batch, z_dim)
        x: torch.tensor, with dimensionality (mini-batch, x_dim)
        """
        if type == 'log_prob':
            x_prob = self.decode(z)
            x_flat = x.view(x.size(0), -1)
            x_prob_flat = x_prob.view(x_prob.size(0), -1)
            return log_bernoulli(x_flat,x_prob_flat,reduction='sum',dim=1) #dim =1. is used to sum the log-probs for every pixel of every record in the batch 
        else:
            return self.sample(z)

Please answer the following questions:

#### Question 3 (0.5 pt)

Please explain your choice of the distribution for image data used in this assignment. Additionally, please write it down mathematically (if you think that presenting it as the log-probability, then please do it).

ANSWER: [Please fill in]

#### Question 4 (0.5 pt)

Please explain how one can sample from the distribution chosen by you. Please be specific and formal (i.e., provide mathematical formulae). If applicable, please provide a code snippet.

### Prior

The prior is the marginal distribution over latent variables, i.e., $p(\mathbf{z})$. It plays a crucial role in the generative process and also in synthesizing images of a better quality.

In this assignment, you are asked to implement a prior that is learnable (e.g., parameterized by neural networks). If you decide to implement the standard Gaussian prior only, then please be aware that you will not get any points.


For the learnable prior you can choose one of the following options:


*   Mixture of Gaussians
*   Normalizing Flow


In [8]:
# YOUR CODE GOES HERE
# NOTES:
# (i) Implementing the standard Gaussian prior does not give you any points!
# (ii) The function "sample" must be implemented.
# (iii) The function "forward" must return the log-probability, i.e., log p(z)

class Prior(nn.Module):
    def __init__(self,latent_space_dim = 20 , num_clusters = 10, device='cpu'):
        super(Prior, self).__init__()
        self.device = device

        self.num_clusters = num_clusters
        self.latent_space_dim = latent_space_dim
        
        # Learnable parameters for the Gaussian Mixture Model (GMM)
        # We have 'num_clusters' different Gaussian components.
        self.means = nn.Parameter(torch.randn(num_clusters,latent_space_dim))
        self.log_vars = nn.Parameter(torch.zeros_like(self.means)) 
        self.w = nn.Parameter(torch.zeros(num_clusters,1,1)) # weights that will be used to calculate π_k from GMM formula.These will be summed to 1 later with softmax


    def sample(self, batch_size):
        # Calculate mixing probabilities π_k for each cluster
        log_probs = F.log_softmax(self.w,dim=0).flatten()
        probs = torch.exp(log_probs)

        indices = torch.multinomial(probs,batch_size,replacement=True)# We sample indices based on the mixing probabilities 'probs'.
        
        # Use the 'indices as a mask to gather the mean and log-variance for the selected cluster for each sample
        means = self.means[indices]
        log_vars = self.log_vars[indices]
        
        #Reparameterization trick: z= mu + s*e
        err = torch.randn_like(means)
        z = means + torch.exp(0.5*log_vars)*err
        return z
    
    def log_prob(self,z):
        # Calculate the log-likelihood of z under the GMM prior:
        log_probs = F.log_softmax(self.w,dim=0).flatten()
        log_probs_list = []

        # Iterate over each cluster component k
        for k in range(self.num_clusters):
            mu_k = self.means[k]
            log_var_k = self.log_vars[k]

            # We calculate the probability of getting z given cluster k 
            # reduction='sum' sums over the latent dimensions (dim=1) to get a single value per sample
            log_prob_k = log_normal_diag(z, mu_k, log_var_k, reduction='sum', dim=1)
            
            # Applying the formula to ge tthe total probability: log(pi_k * N(z|...)) = log(pi_k) + log(N(z|...))
            total_prob_k = log_prob_k + log_probs[k]
            
            log_probs_list.append(total_prob_k)
        final_log_probs = torch.stack(log_probs_list,dim=1)#Stack the results from all clusters: shape (batch_size, num_clusters)
        return torch.logsumexp(final_log_probs,dim=1)# Apply Log-Sum-Exp across the cluster dimension to get final log p(z)

    def forward(self, z):
        return self.log_prob(z)

#### Question 5 (2 pts max)

**Option 1 (0 pt):  Standard Gaussian**

**NOTE: *If you decide to use the standard Gaussian prior, please indicate it in your answer. However, you will get 0 pt for this question.***

**Option 2 (0.5 pt): Mixture of Gaussains**

Please do the following:
- (0.25 pt) Please explain your prior and write it down mathematically
- (0.15 pt) Please write down its sampling procedure (if necessary, please add a code snippet).
- (0.1 pt) Please write down its log-probability (a mathematical formula).

**Option 3 (2 pts): Normalizing Flow**

Please do the following:
- (1 pt) Please explain your prior and write it down mathematically
- (0.5 pt) Please write down its sampling procedure (if necessary, please add a code snippet).
- (0.5 pt) Please write down its log-probability (a mathematical formula).

### Complete VAE

The last class is `VAE` tha combines all components. Please remember that this class must implement the **Negative ELBO** in `forward`, as well as `sample` (*hint*: it is a composition of `sample` functions from the prior and the decoder).

In [9]:
# YOUR CODE GOES HERE
# This class combines Encoder, Decoder and Prior.
# NOTES:
# (i) The function "sample" must be implemented.
# (ii) The function "forward" must return the negative ELBO. Please remember to add an argument "reduction" that is either "mean" or "sum".
class VAE(nn.Module):
    def __init__(self,encoder:Encoder,decoder:Decoder,prior:Prior, device='cpu'):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.prior = prior

    def sample(self, batch_size=64):
        z = self.prior.sample(batch_size=batch_size)
        return self.decoder.sample(z)

    def forward(self, x, reduction='mean'):
        # Please correctly implement this function
        # mu_e, log_var_e = self.encoder.encode(x)
        # z = self.encoder.sample(mu_e=mu_e,log_var_e=log_var_e)
        mu_e, log_var_e = self.encoder.encode(x)
        z = self.encoder.sample(mu_e=mu_e, log_var_e=log_var_e)

        RE = self.decoder.forward(z,x,type='log_prob') # Reconstruction error 

        log_q = log_normal_diag(z, mu_e, log_var_e, reduction='sum', dim=1)
        log_p = self.prior.forward(z)

        KL = log_q-log_p #KL Divergence 


        # Negative ELBO (NELBO)
        NELBO = KL-RE

        if reduction == 'sum':
            return NELBO.sum()
        else:
            return NELBO.mean()

#### Question 6 (0.5 pt)

Please write down mathematically the **Negative ELBO** and provide a code snippet.

ANSWER: [Please fill in]

### Evaluation and training functions

**Please do not remove or modify them.**

In [10]:
# DO NOT REMOVE

def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # EVALUATION
    if model_best is None:
        # load best performing model
        model_best = torch.load(name + '.model')

    model_best.eval()
    loss = 0.
    N = 0.
    for indx_batch, (test_batch, _) in enumerate(test_loader):
        test_batch = test_batch.to(device)
        loss_t = model_best.forward(test_batch, reduction='sum')
        loss = loss + loss_t.item()
        N = N + test_batch.shape[0]
    loss = loss / N

    if epoch is None:
        print(f'FINAL LOSS: nll={loss}')
    else:
        print(f'Epoch: {epoch}, val nll={loss}')

    return loss


def samples_real(name, test_loader, shape=(28,28)):
    # real images-------
    num_x = 4
    num_y = 4
    x, _ = next(iter(test_loader))
    x = x.to('cpu').detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], shape)
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name+'_real_images.pdf', bbox_inches='tight')
    plt.close()


def samples_generated(name, data_loader, shape=(28,28), extra_name=''):
    x, _ = next(iter(data_loader))
    x = x.to('cpu').detach().numpy()

    # generations-------
    model_best = torch.load(name + '.model')
    model_best.eval()

    num_x = 4
    num_y = 4
    x = model_best.sample(num_x * num_y)
    x = x.to('cpu').detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], shape)
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name + '_generated_images' + extra_name + '.pdf', bbox_inches='tight')
    plt.close()


def plot_curve(name, nll_val):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel('epochs')
    plt.ylabel('nll')
    plt.savefig(name + '_nll_val_curve.pdf', bbox_inches='tight')
    plt.close()

In [11]:
# DO NOT REMOVE

def training(name, max_patience, num_epochs, model, optimizer, training_loader, val_loader, shape=(28,28)):
    nll_val = []
    best_nll = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # TRAINING
        model.train()
        for indx_batch, (batch, _) in enumerate(training_loader):
            batch = batch.to(device)
            loss = model.forward(batch, reduction='mean')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        loss_val = evaluation(val_loader, model_best=model, epoch=e)
        nll_val.append(loss_val)  # save for plotting

        if e == 0:
            print('saved!')
            torch.save(model, name + '.model')
            best_nll = loss_val
        else:
            if loss_val < best_nll:
                print('saved!')
                torch.save(model, name + '.model')
                best_nll = loss_val
                patience = 0

                samples_generated(name, val_loader, shape=shape, extra_name="_epoch_" + str(e))
            else:
                patience = patience + 1

        if patience > max_patience:
            break

    nll_val = np.asarray(nll_val)

    return nll_val

### Setup

**NOTE: *Please comment your code! Especially if you introduce any new variables (e.g., hyperparameters).***

In the following cells, we define `transforms` for the dataset. Next, we initialize the data, a directory for results and some fixed hyperparameters.

In [12]:
# PLEASE DEFINE APPROPRIATE TRANFORMS FOR THE DATASET
transforms_train = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

transforms_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

Please do not modify the code in the next cell.

In [13]:
# DO NOT REMOVE
#-dataset
dataset = MNIST('./files/', train=True, download=True,
                      transform=transforms_train
                )

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [50000, 10000], generator=torch.Generator().manual_seed(14))

test_dataset = MNIST('./files/', train=False, download=True,
                      transform=transforms_test
                     )
#-dataloaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#-creating a dir for saving results
name = 'vae'
result_dir = images_dir + 'results/' + name + '/'
if not os.path.exists(result_dir):
    os.makedirs(result_dir, exist_ok=True)

#-hyperparams (please do not modify them for the final report)
num_epochs = 1000 # max. number of epochs
max_patience = 20 # an early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped

In the next cell, please initialize the model. Please remember about commenting your code!

In [14]:
# YOUR CODE COMES HERE:
# Hyperparameters
latent_space_dim = 20      
num_clusters = 10 

#Architecture params
input_channels =1
conv_1_filters = 32
conv_2_filters = 64
kernel_size = 3
stride=2
padding=1
final_dim = 7 # after first stride 28/2 = 14 and then after second stride 14/2 =7


encoder_net = nn.Sequential(
    nn.Conv2d(in_channels=input_channels,
              out_channels=conv_1_filters,
              kernel_size=kernel_size,
              padding=padding,
              stride=stride),
    nn.ReLU(),
    nn.Conv2d(in_channels=conv_1_filters,
              out_channels=conv_2_filters,
              kernel_size=kernel_size,
              padding=padding,
              stride=stride),
    nn.ReLU(),
    #Takes the 64,7,7 input and flattens it out so that the Linear layer can work with it .
    nn.Flatten(),
    nn.Linear(in_features=conv_2_filters*final_dim*final_dim,
              out_features=2*latent_space_dim) 
)

decoder_net = nn.Sequential(
    nn.Linear(in_features=latent_space_dim,
              out_features=conv_2_filters*final_dim*final_dim),
    nn.ReLU(),
    nn.Unflatten(1,torch.Size([conv_2_filters,final_dim,final_dim])),
    nn.ConvTranspose2d(in_channels=conv_2_filters,
                       out_channels=conv_1_filters,
                       kernel_size=kernel_size,
                       padding=padding,
                       output_padding=padding,
                       stride=stride
                       ),
    nn.ReLU(),
    nn.ConvTranspose2d(in_channels=conv_1_filters,
                       out_channels=input_channels,
                       kernel_size=kernel_size,
                       output_padding=padding,
                       padding=padding,
                       stride=stride
                       ),
    
)


encoder = Encoder(encoder_net=encoder_net)
decoder = Decoder(decoder_net=decoder_net)

prior = Prior(latent_space_dim=latent_space_dim, num_clusters=num_clusters, device=device)
model = VAE(encoder, decoder, prior, device=device).to(device)

Please initialize the optimizer

In [15]:
# PLEASE DEFINE YOUR OPTIMIZER
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#### Question 7 (0.5 pt)

Please explain the choice of the optimizer, and comment on the choice of the hyperparameters (e.g., the learing reate value).

ANSWER: [Please fill in]

### Training and final evaluation

In the following two cells, we run the training and the final evaluation.

In [16]:
# DO NOT REMOVE OR MODIFY
# Training procedure
nll_val = training(name=result_dir + name, max_patience=max_patience, 
                   num_epochs=num_epochs, model=model, optimizer=optimizer,
                   training_loader=train_loader, val_loader=val_loader,
                   shape=(28,28))

Epoch: 0, val nll=112.47562062988281
saved!
Epoch: 1, val nll=108.04412717285156
saved!


/var/folders/1m/wkv7c1qj5qv7hrgzg1vx89q40000gn/T/ipykernel_74027/3403963310.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_best = torch.load(name + '.model')


Epoch: 2, val nll=105.75564926757812
saved!
Epoch: 3, val nll=104.29023161621093
saved!
Epoch: 4, val nll=103.28905803222656
saved!
Epoch: 5, val nll=102.8394048828125
saved!
Epoch: 6, val nll=102.2699553466797
saved!
Epoch: 7, val nll=101.73761108398438
saved!
Epoch: 8, val nll=101.63134802246094
saved!
Epoch: 9, val nll=100.95898487548828
saved!
Epoch: 10, val nll=100.7693845703125
saved!
Epoch: 11, val nll=100.73055393066406
saved!
Epoch: 12, val nll=100.61476120605468
saved!
Epoch: 13, val nll=100.40415528564453
saved!
Epoch: 14, val nll=99.94315263671875
saved!
Epoch: 15, val nll=99.70311716308593
saved!
Epoch: 16, val nll=99.60444890136719
saved!
Epoch: 17, val nll=99.69520310058594
Epoch: 18, val nll=99.32866264648437
saved!
Epoch: 19, val nll=99.18696915283203
saved!
Epoch: 20, val nll=98.90877180175781
saved!
Epoch: 21, val nll=98.7026603515625
saved!
Epoch: 22, val nll=98.77105961914063
Epoch: 23, val nll=98.90883354492188
Epoch: 24, val nll=98.65520288085938
saved!
Epoch: 25

In [17]:
# DO NOT REMOVE OR MODIFY
# Final evaluation
test_loss = evaluation(name=result_dir + name, test_loader=test_loader)
f = open(result_dir + name + '_test_loss.txt', "w")
f.write(str(test_loss))
f.close()

samples_real(result_dir + name, test_loader)
samples_generated(result_dir + name, test_loader, extra_name='_FINAL')

plot_curve(result_dir + name, nll_val)

/var/folders/1m/wkv7c1qj5qv7hrgzg1vx89q40000gn/T/ipykernel_74027/3403963310.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_best = torch.load(name + '.model')


FINAL LOSS: nll=94.78668686523437


/var/folders/1m/wkv7c1qj5qv7hrgzg1vx89q40000gn/T/ipykernel_74027/3403963310.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_best = torch.load(name + '.model')


### Results and discussion

After a successful training of your model, we would like to ask you to present your data and analyze it. Please answer the following questions.


#### Question 8 (1 pt)

Please select the real data, and the final generated data and include them in this report. Please comment on the following:
- (0.5 pt) Do you think the model was trained properly by looking at the generations? Please motivate your answer well.
- (0.5 pt) What are potential problems with evaluating a generative model by looking at generated data? How can we evalute generative models (ELBO or NLL do not count as an answer)?

ANSWER: [Please fill in]

#### Question 9 (1.25 pt)

Please include the plot of the negative ELBO. Please comment on the following:
- (0.25 pt) Is the training of your VAE stable or unstable? Why?
- (1 pt) What is the influence of the optimizer on your model? Do the hyperparameter values of the optimizer important and how do they influence the training? Motivate well your answer (e.g., run the script with more than one learning rate and present two plots here).

ANSWER: [Please fill in]

# Grading (10pt)

- Question 0: 3pt
- Question 1: 0.5pt 
- Question 2: 0.25pt 
- Question 3: 0.5pt 
- Question 4: 0.5pt 
- Question 5: 2pt 
- Question 6: 0.5pt
- Question 7: 0.5pt
- Question 8: 1pt
- Question 9: 1.25pt